<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/03_langchain/06_openai_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Function Calling


## Setup

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env' # read local .env file
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_api_key)

import warnings
warnings.filterwarnings("ignore")

## Function Calling

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# Define a list of dicts so the model knows what functions it can call and their argument schema
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

### Pass a message that is related to a function

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
# Call the ChatCompletion endpoint
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions
)

In [ ]:
print(response)

ChatCompletion(id='chatcmpl-CbN14eLwML8j8atO6S6mer5afFasP', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None))], created=1763022646, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=17, prompt_tokens=79, total_tokens=96, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
response_message = response.choices[0].message

In [ ]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)

In [ ]:
content = response.choices[0].message.content
content # this is None as you see above

In [ ]:
response_message.function_call

FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather')

In [ ]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston, MA'}

In [ ]:
args = json.loads(response_message.function_call.arguments)

In [ ]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

The model correctly understood that it should use the function instead of generating its own answer, and it also correctly produced the required arguments. By passing these arguments to the function locally, we execute it and obtain the actual result.

### Pass a message that is not related to a function.

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
)

In [ ]:
print(response)

ChatCompletion(id='chatcmpl-CbN7xRi7QGcTzIdcn5KYnYZlcSK02', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763023073, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=10, prompt_tokens=74, total_tokens=84, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


Notice what the model answers and also that it understands not to use the function.

In [ ]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [ ]:
print(response.choices[0].message.function_call)

None


### Pass additional parameters to force the model to use or not a function

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto", # llm chooses in this case
)
print(response)

ChatCompletion(id='chatcmpl-CbND72qXkDSoIvtD6VygzCWc1fyL9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763023393, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=10, prompt_tokens=74, total_tokens=84, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [ ]:
print(response.choices[0].message.function_call)

None


Setting ```function_call="auto"``` explicitly lets the model decide whether to call a function or respond with text. In messages like ```"hi!"```, the model correctly responds directly, but for requests that match a function's purpose, it will produce a function_call automatically.

### Use mode 'none' for function call

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-CbNF6AWdjaB8QDoi0oCAYlt4QNhgW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763023516, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=9, prompt_tokens=75, total_tokens=84, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [ ]:
print(response.choices[0].message.function_call)

None


In this case, setting function_call="none" forces the model not to call any functions, no matter what the user asks. So even if the user’s message could match a function, the model will always respond with plain text.

### When the message should call a function and still uses mode 'none'

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-CbNGxjJGXEm1MpH0BojorbQIX1ggs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Please hold on a moment while I get the current weather for Boston.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763023631, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=14, prompt_tokens=79, total_tokens=93, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

Please hold on a moment while I get the current weather for Boston.


In [ ]:
print(response.choices[0].message.function_call)

None


### Force calling a function

In [ ]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-CbNHUt6RCv8gzV9HofmcZD00UsufA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"New York, NY"}', name='get_current_weather'), tool_calls=None))], created=1763023664, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=8, prompt_tokens=84, total_tokens=92, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

None


In [ ]:
print(response.choices[0].message.function_call)

FunctionCall(arguments='{"location":"New York, NY"}', name='get_current_weather')


### Final notes.

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-CbNMPYw845VtGyxVPsRHIfQyde0Hk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None))], created=1763023969, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=7, prompt_tokens=89, total_tokens=96, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

None


In [ ]:
print(response.choices[0].message.function_call)

FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather')


In [ ]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

In [ ]:
print(observation)

{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


In [ ]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(response)

ChatCompletion(id='chatcmpl-CbNOc7KI9URV19R3T7zoSqFeQkEqI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The weather in Boston is currently 72°F, with sunny and windy conditions.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763024106, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=16, prompt_tokens=56, total_tokens=72, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(response.choices[0].message.content)

The weather in Boston is currently 72°F, with sunny and windy conditions.


In this notebook, we explored step by step how a model (LLM) determines when to use a function for different user prompts. In this implementation, we manually extracted the arguments generated by the model, passed them to the corresponding function, and then fed the function's output back to the LLM to produce the final natural-language answer.


**Notes**:
- LLMs don't always produce the same results.